### Import

In [ ]:
from langchain_core.language_models.fake_chat_models import (
    FakeListChatModel, 
)
from langchain_core.runnables import ConfigurableField, Runnable
from langchain_core.output_parsers import (
    JsonOutputParser, 
    StrOutputParser, 
    CommaSeparatedListOutputParser,
    NumberedListOutputParser,
    PydanticOutputParser,
)
import json
from pydantic import (
    BaseModel,
    Field
)

In [ ]:
def print_dict(val: dict):
    print(json.dumps(val, indent=4))

### Test Model

In [ ]:
resp_list = [
    'Hello from AI', 
    'How may I help you?', 
    'Another message from AI']
model = FakeListChatModel(responses = resp_list)

In [ ]:
resp = model.invoke('')
print(resp.__class__)
print(resp.content)
for resp in resp_list[1:]:
    print(model.invoke('').content)

In [ ]:
model2 = model.configurable_fields(
    responses=ConfigurableField(
        id="responses",
        name="List of responses to cycle through in order.",
        description="List of responses to cycle through in order.",
    )
)

In [ ]:
custom_resp = ['Custom AI message']
config = {"configurable": {"responses": custom_resp}}
resp = model2.invoke("", config=config)
print(resp.__class__)
print(resp.content)

### With Parser

In [ ]:
chain = model | StrOutputParser()
resp = chain.invoke('')
print(resp.__class__)
print(resp)

for resp in resp_list[1:]:
    print(chain.invoke(''))

In [ ]:
custom_resp = '{"price": "$1,000", "RAM": "6GB"}'
# custom_resp = json.dumps({'price': '$1,000', 'RAM': '6GB'})
config = {"configurable": {"responses": [custom_resp]}}
parser = JsonOutputParser()
print(parser.get_format_instructions())

chain2 = model2 | parser
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

In [ ]:
custom_resp = 'one, two, three'
config = {"configurable": {"responses": [custom_resp]}}
parser = CommaSeparatedListOutputParser()
print(parser.get_format_instructions())

chain2 = model2 | parser
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

In [ ]:
custom_resp = """
1. One
2. Two
3. Three
"""
config = {"configurable": {"responses": [custom_resp]}}
parser = NumberedListOutputParser()
print(parser.get_format_instructions())

chain2 = model2 | parser
resp = chain2.invoke("", config=config)
print(resp.__class__)
print(resp)

### Pydantic Parser

In [ ]:
class AnswerWithJustification(BaseModel):
    '''An answer to the user question along with justification for the answer.'''
    answer: str
    justification: str

In [ ]:
data = {
    'answer': 'They weigh the same',
    'justification': (
        'Both a pound of bricks and a pound of feathers weigh one pound. '
        'The weight is the same, but the volume and density of the two substances differ.'
    )
}
config = {"configurable": {"responses": [json.dumps(data)]}}
parser = PydanticOutputParser(pydantic_object=AnswerWithJustification)
print(parser.get_format_instructions())

chain2 = model2 | parser
resp = chain2.invoke("", config=config)
print(resp.__class__)
print_dict(resp.__dict__)